# Some Tips for JupyterNotebook : 

-Scroll up and down your cells with your Up and Down keys.

-Press A or B to insert a new cell above or below the active cell.

-M will transform the active cell to a Markdown cell.

-Y will set the active cell to a code cell.

-D + D (D twice) will delete the active cell.

-Z will undo cell deletion.

-Hold Shift and press Up or Down to select multiple cells at once. With multiple cells selected, Shift + M will merge your selection.

# Some tests for text and running code 


In [ ]:
print("hello world !")

In [ ]:
x = 3
for i in range(0, 5):
    print(i + x) 

Salut text intermediaire 

In [ ]:
import time 
time.sleep(3)

Salut 

Je tape ce texte pour tester le Markdown 

Ca marche bien !

In [ ]:
string = "Bonjour comment ca va ? "
print(string)

# Algorithm(s) used for the AI part of the app 


In [ ]:
max_score = 10
min_score = 0
critical = 0

def input_score(string, list_input):
    score = int(input("Rentrez votre " + string + "\t"))
    list_input.append(score)
    return score

# max_score_is_bad is a boolean
def find_factor(score,name, epsilon, max_score_is_bad):
    if (max_score_is_bad): 
        if (abs(max_score - score) < epsilon):
            print(name + " est un facteur important du modele")
    else :
        if (score < epsilon):
            print(name + " est un facteur important du modele")


input_list = []
age     = input_score("age", input_list)
fatigue = input_score("fatigue", input_list)
douleur = input_score("douleur", input_list)
sommeil = input_score("sommeil", input_list)

del input_list[0] # we don t consider the age as an input for this time 
    
print(input_list)

# 1st exemple 
if ((max_score - fatigue) < 2 or sommeil < 2 or (max_score - douleur) < 2):
    critical = 1
    print("You should try to do this or that ")
elif (fatigue > 2 and fatigue < 5):
    print("Keep going it s better")
else : 
    print("Great job you seems to be good ! ")
    
# 2nd exemple 
for score in input_list:
    if score < 2:
        critical = 1
        print("You should try to do this or that #2")
        
        
        
# do the correlation between the age and the (back) pain for example
if (douleur > 7):
    if (age > 60):
        print("Try to stretch yourself ")
    else:
        print("Change your position when you are working or whatever")

find_factor(fatigue, "fatigue", 2, 1)
find_factor(douleur, "douleur", 2, 1)
find_factor(sommeil, "sommeil", 2, 0)

# CLUSTER TEST ON DATASET

# CREATE THE DATASET 

In [76]:
import numpy as np
import pandas as pd
import names
import gender_guesser.detector as gender


#defining a seed

np.random.seed(42)

detector = gender.Detector()

#generating first col
names_col = [names.get_first_name() for i in range(800)]

#names
df=pd.DataFrame(data=names_col,
                columns=["name"])

sexe_col = [0 if (detector.get_gender(names_col[i]) == "male" or detector.get_gender(names_col[i]) == "mostly_male")
            else 1 for i in range(800)]

#sexes
df1 = pd.DataFrame(data=sexe_col,
                  columns=["sexe"])

#age 
df2 = pd.DataFrame(np.random.randint(0, 100, size=800),
                  columns=["age"])

#pain
df3 = pd.DataFrame(np.random.randint(0, 11, size=800),
                  columns=["pain"]) 

#sleep
df4 = pd.DataFrame(np.random.randint(0, 11, size=800),
                  columns=["sleep"])

df = pd.concat([df, df1, df2, df3, df4], axis=1)


final_tab = df


final_tab=df[["name", "sexe", "age", "pain", "sleep"]]


X = final_tab.sample(frac=1)
X.sample(frac=1).to_csv("test1.csv",index=False)

In [77]:
file = pd.read_csv("test1.csv")
file.head(10)

,name,sexe,age,pain,sleep
0,Jason,0,8,8,0
1,George,0,64,9,10
2,Michael,0,32,7,5
3,Michael,0,32,4,5
4,Chad,0,65,7,6
5,Patricia,1,1,10,1
6,Robert,0,31,0,4
7,Lorena,1,69,10,8
8,Clara,1,57,7,1
9,Helen,1,44,5,2


In [78]:
file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    800 non-null    object
 1   sexe    800 non-null    int64 
 2   age     800 non-null    int64 
 3   pain    800 non-null    int64 
 4   sleep   800 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 31.4+ KB


In [79]:
import plotly.express as plot

fig = plot.scatter_matrix(file,
                          width=1200, height=1600)
fig.show()

In [85]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go


X=df.drop(["name"],axis=1)
scaler = MinMaxScaler()
scaler.fit(X)
X=scaler.transform(X)
inertia = []
for i in range(1,11):
    kmeans = KMeans(
        n_clusters=i, init="k-means++",
        n_init=10,
        tol=1e-04, random_state=42
    )
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)
fig = go.Figure(data=go.Scatter(x=np.arange(1,11),y=inertia))
fig.update_layout(title="Inertia vs Cluster Number",xaxis=dict(range=[0,11],title="Cluster Number"),
                  yaxis={'title':'Inertia'},
                 annotations=[
        dict(
            x=3,
            y=inertia[2],
            xref="x",
            yref="y",
            text="Elbow!",
            showarrow=True,
            arrowhead=7,
            ax=20,
            ay=-40
        )
    ])